- collections：数据集
- 拷贝、副本：不修改原数据
- view：修改原数据（相当于在原数据上操作）

## pandas入门

#### pandas的数据结构介绍

In [1]:
import numpy as np
from pandas import Series, DataFrame

import pandas as pd

In [2]:
obj = Series([4,7, -5, 3])

obj

0    4
1    7
2   -5
3    3
dtype: int64

In [3]:
obj.values

array([ 4,  7, -5,  3])

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
# 指定标记索引
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2['a']
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [7]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    4
dtype: int64

In [8]:
obj2[obj2>0] # 只是一个副本不修改原来的数据

d    4
b    7
c    3
dtype: int64

In [9]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

## Series其实可以被看做一个定长的有序字典，一个索引值到数据值的映射

通过Hashtable实现

In [10]:
# 通过Python字典创建Series

sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = Series(sdata)
obj3 # key 会有序排列

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [11]:
# 根据 sdata 创建一个对象，修改索引
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)

obj4 # 'California 没有匹配项而变成NAN

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [12]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

#### Series在运算中可以自动对齐不同的索引数据

In [13]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

In [14]:
# Series对象本身及其索引都有一个name属性
obj4.name = 'population'
obj4.index.name = 'state'

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [15]:
# 修改索引
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

创建DataFrame

In [16]:
# dataframe 构建方法有很多，常用的是传入一个
#  由等长列表或Numpy数组组成的字典
# 即传入的是一个字典
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
       'year':[2000, 2001, 2002, 2001, 2002],
       'pop':[1.5, 1.7, 3.6, 2.4, 2.9]}
frame = DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


In [17]:
# 如果在构建是指定了 列的顺序，则会按照指定顺序进行列排序
DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


In [18]:
# 和Series一样，如果传入的列索引在数据中找不到，就会产生NA值
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                  index = ['one', 'two', 'three', 'four', 'five'])

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [19]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [20]:
# 通过类似字典的方式，可以将DataFrame的列获取为一个Series
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
Name: state, dtype: object

In [21]:
# DataFrame 的行也可以通过位置或名称获取，比如索引ix
frame2.ix['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

In [22]:
# 修改某一列的值
frame2['debt'] = 16.5

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


In [23]:
frame2['debt'] = np.arange(5.)

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0


#### 将列表或数组赋值给某个列时，长度必须跟DataFrame长度匹配。如果赋值是一个Series，就会精确匹配DataFrame的索引，所有空位会被填上NA值。

In [24]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'six'])

frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,NaN


In [25]:
# 为不存在的列赋值会创建一个新列。 del用于删除列
frame2['eastern'] = frame2.state == 'Ohio'

frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,NaN,False


In [26]:
del frame2['eastern']

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,NaN


#### 通过索引返回的是视图，对返回的Series做的任何修改都会反映到原DataFrame上。通过Series的copy方法即可显示地复制列。

## 另一种是使用嵌套字典创建DataFrame

#### 这样，外层字典的键作为列，内层键则作为行索引

In [27]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
      'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [28]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [29]:
DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


In [30]:
pdata = {'Ohio': frame3['Ohio'][:-1],
        'Nevada': frame3['Nevada'][:2]}
pdata # frame3 前两行

{'Nevada': 2000    NaN
 2001    2.4
 Name: Nevada, dtype: float64, 'Ohio': 2000    1.5
 2001    1.7
 Name: Ohio, dtype: float64}

In [31]:
DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


In [32]:
frame3.index.name = 'year'; frame3.columns.name = 'state'

frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [33]:
# 和Series一样，values属性也会以二维ndarray的形式返回DataFrame中的数据
frame3.values

array([[ nan,  1.5],
       [ 2.4,  1.7],
       [ 2.9,  3.6]])

In [34]:
# 但是如果DataFrame各列的数据类型不同，
# 则数组的数据类型就会选用能兼容所有列的数据类型
# 比如 list
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, nan]], dtype=object)

#### pandas, Series组成有索引对象，用于管理轴标签和其他元数据（比如轴名称等）。

Index对象是不可修改的，immutable

这样才能使Index对象在多个数据结构之间安全共享

In [35]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index

index

Index(['a', 'b', 'c'], dtype='object')

In [36]:
index[1:]

Index(['b', 'c'], dtype='object')

In [37]:
# 除了长得像数组，Index的功能也类似一个固定大小的集合
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [38]:
'Ohio' in frame3.columns

True

In [39]:
2003 in frame3.index

False

## 基本功能

#### 重新索引

根据新的索引值对Series、DataFrame重排，如果某个索引值当前不存在，就引入缺失值。

In [40]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [41]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [42]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

In [43]:
# 对于时间序列数据，重新索引时可能需要做一些插值处理，
# method选项可达这样的目的。使用ffill实现前向值填充

obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [44]:
# 对于DataFrame，reindex可以修改索引（行）、列，或两个都修改。
# 如果仅传入一个序列，则会重新索引行
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                 columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [45]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [46]:
# 重排索引列，使用columns关键字
states = ['Texas', 'Utah', 'California']

frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [47]:
# 也可以同时对列和行进行重新索引，插值只能按行应用（即轴0）
frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill', columns=states)

,Texas,Utah,California
a,1,NaN,2
b,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [48]:
# 若想使重新索引更加简洁，使用ix标签索引功能
frame.ix[['a', 'b', 'c', 'd'], states]

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


#### 丢弃指定轴上的项

In [49]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')

new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [50]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [51]:
# 对于DataFrame，可以删除任意轴上的索引值
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])

data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [52]:
data.drop('two', axis=1) #指定轴，如果没有ValueError

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


#### 索引、选取和过滤

但是利用标签的切片运算与普通Python的切片运算不同，末端使包含的，inclusive

In [53]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [54]:
obj[1:2] # exclusive

b    1.0
dtype: float64

In [55]:
obj['b','c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [56]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [57]:
data[data['three']>5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [58]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


#### 上面的都是对列索引，为了实现对行索引，引入：索引字段ix

In [59]:
data.ix['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

In [60]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


In [61]:
data.ix[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [62]:
type(data.ix[0])

pandas.core.series.Series

In [63]:
data.ix['Utah', 'two']

9

In [64]:
data.get_value(index='Utah', col='two')

9

#### 算术运算和数据对齐

In [65]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [66]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [67]:
# 对于DataFrame，对齐操作会同时发生在行和列上
df1 = DataFrame(np.arange(9).reshape(3, 3), columns=list('bcd'),
               index=['Ohio', 'Texas', 'Colorado'])

df2 = DataFrame(np.arange(12).reshape(4, 3), columns=list('bde'),
               index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1;df2

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [68]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


#### 在算术方法中填充值

In [69]:
df1 = DataFrame(np.arange(12).reshape(3, 4), columns=list('abcd'))
df2 = DataFrame(np.arange(20).reshape(4, 5), columns=list('abcde'))
df1

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [70]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [71]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [72]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0


#### DataFrame和Series之间的运算

先看一个numpy二维数组与其某行之间的差

In [73]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [74]:
arr - arr[0]

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.]])

In [75]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.ix[0]
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [76]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [77]:
# 默认情况下，DataFrame和Series之间的算术运算会将series的索引匹配到DataFrame的列，
# 然后沿着行一直向下广播

frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [78]:
series2 = Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [79]:
# 如果某个索引值在DataFrame中找不到，则参与运算的两个对象就会被重新索引
# 以形成并集
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [80]:
# 如果希望匹配行在列上广播，则必须使用算术运算方法
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [81]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [85]:
frame.sub(series3, axis=0) # 匹配行，且在列上广播

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


In [86]:
frame.sub(series3)

,Ohio,Oregon,Texas,Utah,b,d,e
Utah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ohio,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oregon,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# 函数应用和映射
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])

frame

,b,d,e
Utah,0.522741,-0.072365,-0.580669
Ohio,0.579944,0.516823,0.430106
Texas,1.758867,0.793427,0.351199
Oregon,0.895868,0.315603,-0.016203


In [88]:
np.abs(frame)

,b,d,e
Utah,0.522741,0.072365,0.580669
Ohio,0.579944,0.516823,0.430106
Texas,1.758867,0.793427,0.351199
Oregon,0.895868,0.315603,0.016203


In [89]:
f = lambda x: x.max() - x.min()

frame.apply(f)

b    1.236126
d    0.865791
e    1.010775
dtype: float64

In [90]:
frame.apply(f, axis=1)

Utah      1.103411
Ohio      0.149838
Texas     1.407667
Oregon    0.912071
dtype: float64

In [91]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f)

,b,d,e
min,0.522741,-0.072365,-0.580669
max,1.758867,0.793427,0.430106


In [92]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Utah,0.52,-0.07,-0.58
Ohio,0.58,0.52,0.43
Texas,1.76,0.79,0.35
Oregon,0.90,0.32,-0.02


In [93]:
# 之所以叫 applymap 是因为 Series 有一个 map 函数
frame['e'].map(format)

Utah      -0.58
Ohio       0.43
Texas      0.35
Oregon    -0.02
Name: e, dtype: object

In [94]:
## 排序和排名
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [95]:
# 对于DataFrame，可以根据任意一个轴上的索引进行排序
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                 columns=['d', 'a', 'b', 'c'])

frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [96]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [100]:
obj = Series([4, 7, -3, 2])

# obj.order()
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

In [102]:
# 任何缺失值默认放到Series的末尾
obj = Series([4, np.nan, 7, np.nan, -3, 2])

In [103]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [104]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2


In [106]:
frame.sort_values(by='b')

,a,b
2,0,-3
3,1,2
0,0,4
1,1,7


In [107]:
# 根据多个列进行排序
frame.sort_values(by=['a', 'b'])

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


In [109]:
# rank 类似于 非参检验那种排名
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj.rank() # 默认保持平级关系

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [110]:
# 也可以按大小从1排名， 破坏平级关系
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [112]:
obj.rank(method='max')

0    7.0
1    1.0
2    7.0
3    5.0
4    3.0
5    2.0
6    5.0
dtype: float64

In [113]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

In [114]:
# 有重复索引
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])

obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [115]:
obj.index.is_unique

False

In [116]:
# 如果某个索引存在多个，则返回一个Series，否则返回一个标量值
obj['a']

a    0
a    1
dtype: int64

In [118]:
obj['c']

4

In [119]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,0.141912,-0.362400,0.230331
a,-0.670058,-1.800111,1.148199
b,0.284459,0.248021,0.898396
b,0.481985,-0.332969,-0.620318


In [121]:
df.ix['b']

,0,1,2
b,0.284459,0.248021,0.898396
b,0.481985,-0.332969,-0.620318


## 层次化索引

#### 这是一个重要功能，使你能在一个轴上拥有多个索引级别

In [122]:
data = Series(np.random.randn(10),
             index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                    [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1   -0.329482
   2    1.410264
   3    0.081321
b  1    0.099562
   2   -0.783733
   3   -1.073445
c  1    0.948701
   2   -0.869381
d  2    1.769489
   3    0.790806
dtype: float64

In [123]:
data.index # labels中，第一个中的0表示levels中第一个列表的第一个元素

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [124]:
data['b']

1    0.099562
2   -0.783733
3   -1.073445
dtype: float64

In [125]:
data['b':'c']

b  1    0.099562
   2   -0.783733
   3   -1.073445
c  1    0.948701
   2   -0.869381
dtype: float64

In [126]:
data.ix[['b', 'd']]

b  1    0.099562
   2   -0.783733
   3   -1.073445
d  2    1.769489
   3    0.790806
dtype: float64

In [127]:
data[['b', 'd']]

b  1    0.099562
   2   -0.783733
   3   -1.073445
d  2    1.769489
   3    0.790806
dtype: float64

In [128]:
# 在内层直接选取
data[:, 2]

a    1.410264
b   -0.783733
c   -0.869381
d    1.769489
dtype: float64

In [129]:
# 层次话索引 变 DataFrame
data.unstack()

,1,2,3
a,-0.329482,1.410264,0.081321
b,0.099562,-0.783733,-1.073445
c,0.948701,-0.869381,NaN
d,NaN,1.769489,0.790806


In [131]:
data.unstack().stack()

a  1   -0.329482
   2    1.410264
   3    0.081321
b  1    0.099562
   2   -0.783733
   3   -1.073445
c  1    0.948701
   2   -0.869381
d  2    1.769489
   3    0.790806
dtype: float64

In [133]:
# 对于一个DataFrame 每个轴都可以有分层索引
frame = DataFrame(np.arange(12).reshape((4,3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [134]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [135]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [136]:
frame.index

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [139]:
# 重调某条轴上各级别的顺序
# 或根据指定级别上的值对数据进行排序
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [140]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [145]:
frame.sortlevel(1) # 根据index的第二层大小排序，也就是1级别

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [143]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [144]:
frame.swaplevel(0, 1).sortlevel(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [146]:
# 根据级别求和
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [147]:
frame.sum(level='color', axis=1) # 其实使用了pandas的groupby功能

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [148]:
## 使用DataFrame的列
frame = DataFrame({'a': range(7), 'b': range(7, 0 , -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [149]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [150]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [151]:
# reset_index 则相反，还原回去
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 其他有关pandas的话题

#### 整数索引

In [152]:
ser = Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [153]:
## 这时候使用 ser[-1] 就会有歧义，有0,1,2 很难推断
## 而非整数索引则没有
ser2 = Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

In [154]:
ser.ix[:1]

0    0.0
1    1.0
dtype: float64

In [157]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3.iloc(2)

In [159]:
frame = DataFrame(np.arange(6).reshape((3, 2)), index=[2, 0, 1])
frame.iloc(0)

In [166]:
## 面板数据
from pandas_datareader import data as web

pdata = pd.Panel(dict((stk, web.get_data_yahoo(stk, '1/1/2009', '6/1/2012'))
                      for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))

## Panel 中的每一项（类似于DataFrame的列）都是一个DataFrame

In [167]:
pdata

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 868 (major_axis) x 6 (minor_axis)
Items axis: AAPL to MSFT
Major_axis axis: 2009-01-02 00:00:00 to 2012-06-01 00:00:00
Minor_axis axis: Open to Adj Close

In [168]:
pdata = pdata.swapaxes('items', 'minor')
pdata['Adj Close']

,AAPL,DELL,GOOG,MSFT
Date,,,,
2009-01-02,11.808505,10.39902,160.499779,16.501303
2009-01-05,12.306869,10.26359,163.861421,16.655521
2009-01-06,12.103880,10.68922,166.863420,16.850323
2009-01-07,11.842336,10.78596,160.844427,15.835732
2009-01-08,12.062241,10.90204,162.432840,16.330853
2009-01-09,11.786384,10.75694,157.377899,15.843849
2009-01-12,11.536551,10.30228,156.189080,15.803264
2009-01-13,11.412936,10.40869,157.003261,16.087350
2009-01-14,11.103247,9.97338,150.334921,15.494829


In [169]:
pdata.ix[:, '6/1/2012', :]

,Open,High,Low,Close,Volume,Adj Close
AAPL,569.159996,572.650009,560.520012,560.989983,130246900.0,72.996726
DELL,12.150000,12.300000,12.045000,12.070000,19397600.0,11.675920
GOOG,571.790972,572.650996,568.350996,570.981000,6138700.0,285.205295
MSFT,28.760000,28.959999,28.440001,28.450001,56634300.0,25.093451


In [172]:
pdata.ix['Adj Close', '5/22/2012':, :]

,AAPL,DELL,GOOG,MSFT
Date,,,,
2012-05-22,72.473644,14.58765,300.100412,26.248896
2012-05-23,74.241986,12.08221,304.426106,25.675584
2012-05-24,73.560156,12.04351,301.528978,25.640302
2012-05-25,73.165884,12.05319,295.470050,25.631482
2012-05-28,NaN,12.05319,NaN,NaN
2012-05-29,74.464493,12.24666,296.873645,26.072491
2012-05-30,75.362333,12.14992,293.821674,25.878448
2012-05-31,75.174961,11.92743,290.140354,25.746145
2012-06-01,72.996726,11.67592,285.205295,25.093451


In [171]:
stacked = pdata.ix[:, '5/30/2012':, :].to_frame()
stacked

Open        High         Low       Close       Volume  \
Date       minor                                                                
2012-05-30 AAPL   569.199997  579.989990  566.559990  579.169998  132357400.0   
           DELL    12.590000   12.700000   12.460000   12.560000   19787800.0   
           GOOG   588.161028  591.901014  583.530999  588.230992    3827600.0   
           MSFT    29.350000   29.480000   29.120001   29.340000   41585500.0   
2012-05-31 AAPL   580.740021  581.499985  571.460022  577.730019  122918600.0   
           DELL    12.530000   12.540000   12.330000   12.330000   19955600.0   
           GOOG   588.720982  590.001032  579.001013  580.860990    5958800.0   
           MSFT    29.299999   29.420000   28.940001   29.190001   39134000.0   
2012-06-01 AAPL   569.159996  572.650009  560.520012  560.989983  130246900.0   
           DELL    12.150000   12.300000   12.045000   12.070000   19397600.0   
           GOOG   571.790972  572.650996  568.350996  570.981000    6138700.0   
           MSFT    28.760000   28.959999   28.440001   28.450001   56634300.0   

                   Adj Close  
Date       minor              
2012-05-30 AAPL    75.362333  
           DELL    12.149920  
           GOOG   293.821674  
           MSFT    25.878448  
2012-05-31 AAPL    75.174961  
           DELL    11.927430  
           GOOG   290.140354  
           MSFT    25.746145  
2012-06-01 AAPL    72.996726  
           DELL    11.675920  
           GOOG   285.205295  
           MSFT    25.093451

In [173]:
stacked.to_panel()

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3 (major_axis) x 4 (minor_axis)
Items axis: Open to Adj Close
Major_axis axis: 2012-05-30 00:00:00 to 2012-06-01 00:00:00
Minor_axis axis: AAPL to MSFT